In [3]:
import numpy as np
import pandas as pd
import itertools
import time
from random import random
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import (MultiprocessingEvaluator, 
                           Scenario, Constraint, Policy,
                           ScalarOutcome)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.em_framework.evaluators import (perform_experiments, optimize,
                                                   BaseEvaluator)
from ema_workbench.util import ema_logging, save_results, load_results
from ema_workbench.analysis import (pairs_plotting, prim, 
                                    feature_scoring, parcoords,
                                    dimensional_stacking)

from problem_formulation import get_model_for_problem_formulation
%matplotlib inline

ema_logging.log_to_stderr(ema_logging.INFO)

BaseEvaluator.reporting_frequency = 0.01

In [4]:
dike_model,planning_steps = get_model_for_problem_formulation(5)

In [3]:
MORO_levers = load_results('Solutions6000nfe60scen[0.01,0.01,0.01,0.01,0.01]eps.tar.gz')
MORDM_pol,outcomes = MORDM_levers
MORDM_pol = MORDM_pol.sample(n=3)
MORDM_pol = MORDM_pol.iloc[:,0:21]
#results_df = pd.DataFrame.from_dict(_) 

[MainProcess/INFO] results loaded succesfully from C:\Users\user\EPA1361\fin_ass\Runs on sollution\scenario1 MORDMruns_worst case scenario.tar.gz


In [6]:
MORO_res= pd.read_csv('moro_pol.csv')
#MORO_res = MORO_res.sample(n=3)
#MORO_pol = MORO_res.iloc[:,1:22]

In [11]:
MORO_res = MORO_res.drop(['Column1','Total Dikes','RfR'],axis=1)

In [13]:
MORO_pol= MORO_res.iloc[:,0:21]
MORO_pol.columns

Index(['0_RfR 0', '0_RfR 1', '1_RfR 0', '1_RfR 1', '2_RfR 0', '2_RfR 1',
       '3_RfR 0', '3_RfR 1', '4_RfR 0', '4_RfR 1', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.2_DikeIncrease 0',
       'A.2_DikeIncrease 1', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.5_DikeIncrease 0',
       'A.5_DikeIncrease 1'],
      dtype='object')

In [16]:
fin_levers = MORO_pol
fin_levers = fin_levers.reset_index()
fin_levers = fin_levers.rename(index={0:'policy1',1:'policy2',2:'policy3',3:'policy4',
                                      })
fin_levers = fin_levers.drop(['index'],axis=1)

In [17]:
policies = []
pol0 = {}
for key in dike_model.levers:
    pol0.update({key.name:0})                
policy0 = Policy('Policy 0', **pol0)
policies.append(policy0)
for i,lever in enumerate(fin_levers.index):
    policy = fin_levers.loc[lever].to_dict()
    policy = Policy(fin_levers.index[i], **policy)
    policies.append(policy)

In [18]:
policies

[Policy({'0_RfR 0': 0, '0_RfR 1': 0, '1_RfR 0': 0, '1_RfR 1': 0, '2_RfR 0': 0, '2_RfR 1': 0, '3_RfR 0': 0, '3_RfR 1': 0, '4_RfR 0': 0, '4_RfR 1': 0, 'EWS_DaysToThreat': 0, 'A.1_DikeIncrease 0': 0, 'A.1_DikeIncrease 1': 0, 'A.2_DikeIncrease 0': 0, 'A.2_DikeIncrease 1': 0, 'A.3_DikeIncrease 0': 0, 'A.3_DikeIncrease 1': 0, 'A.4_DikeIncrease 0': 0, 'A.4_DikeIncrease 1': 0, 'A.5_DikeIncrease 0': 0, 'A.5_DikeIncrease 1': 0}),
 Policy({'0_RfR 0': 1, '0_RfR 1': 1, '1_RfR 0': 1, '1_RfR 1': 0, '2_RfR 0': 1, '2_RfR 1': 1, '3_RfR 0': 1, '3_RfR 1': 1, '4_RfR 0': 0, '4_RfR 1': 0, 'EWS_DaysToThreat': 0, 'A.1_DikeIncrease 0': 6, 'A.1_DikeIncrease 1': 0, 'A.2_DikeIncrease 0': 9, 'A.2_DikeIncrease 1': 0, 'A.3_DikeIncrease 0': 10, 'A.3_DikeIncrease 1': 0, 'A.4_DikeIncrease 0': 9, 'A.4_DikeIncrease 1': 2, 'A.5_DikeIncrease 0': 10, 'A.5_DikeIncrease 1': 0}),
 Policy({'0_RfR 0': 0, '0_RfR 1': 0, '1_RfR 0': 0, '1_RfR 1': 0, '2_RfR 0': 1, '2_RfR 1': 1, '3_RfR 0': 1, '3_RfR 1': 1, '4_RfR 0': 0, '4_RfR 1': 0, '

In [19]:
n_scenarios = 2000

start = time.time()
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies = policies)
    end = time.time()
print(round((end - start)/3600, 2), "Hours")
save_results(results, "MORO4pol.tar.gz")

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 5 policies * 1 model(s) = 10000 experiments
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] 3000 cases completed
[MainProcess/INFO] 4000 cases completed
[MainProcess/INFO] 5000 cases completed
[MainProcess/INFO] 6000 cases completed
[MainProcess/INFO] 7000 cases completed
[MainProcess/INFO] 8000 cases completed
[MainProcess/INFO] 9000 cases completed
[MainProcess/INFO] 10000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


0.5 Hours


[MainProcess/INFO] results saved successfully to C:\Users\pvbia\OneDrive\Documentos\GitHub\MBDM-group-11\final assignment\MORO4pol.tar.gz
